In [577]:
import pandas as pd

In [598]:
pd.set_option('display.max_rows', None)

In [579]:
df_all = pd.read_csv('../data/df_all.csv').drop(['Unnamed: 0', 'id', 'cluster', 'sentiment'], axis=1)

In [580]:
df_all

,question,answer,count
0,Что вы сможете использовать в своей работе?,"формулировать проблему, 5 поче",1.0
1,Что вы сможете использовать в своей работе?,поиск потерь,1.0
2,Что вы сможете использовать в своей работе?,правило 5 «почему» :),1.0
3,Что вы сможете использовать в своей работе?,умение работать в команде,1.0
4,Что вы сможете использовать в своей работе?,картирование и анализ,1.0
...,...,...,...
29659,•Где вы хотите провести следующую HR-сессию?,в москве,1.0
29660,•Где вы хотите провести следующую HR-сессию?,в сочи,1.0
29661,•Где вы хотите провести следующую HR-сессию?,сочи,1.0
29662,•Где вы хотите провести следующую HR-сессию?,окбм,1.0


In [581]:
index = pd.MultiIndex.from_arrays(df_all.iloc[:, :-1].values.T, names=('question', 'answer'))
df_grouped = pd.DataFrame({'count' : df_all.iloc[:, -1:].values.T[0]}, index=index)

In [582]:
df_grouped

count
question                                     answer                               
Что вы сможете использовать в своей работе?  формулировать проблему, 5 поче    1.0
                                             поиск потерь                      1.0
                                             правило 5 «почему» :)             1.0
                                             умение работать в команде         1.0
                                             картирование и анализ             1.0
...                                                                            ...
•Где вы хотите провести следующую HR-сессию? в москве                          1.0
                                             в сочи                            1.0
                                             сочи                              1.0
                                             окбм                              1.0
                                             сочи                              1.0

[29664 rows x 1 columns]

In [583]:
def get_abbreviations(f):
    abbreviations = dict()
    with open(f) as file:
        for line in file.readlines():
            key, value = map(str.strip, line.split('->'))
            abbreviations[key] = value
    return abbreviations

In [584]:
def replace_abbreviations(sentence):
    abbreviations = get_abbreviations('../data/слова.txt')
    words = sentence.lower().split()
    for i in range(len(words)):
        if words[i] in abbreviations:
            words[i] = abbreviations[words[i]]
    return ' '.join(words)

In [585]:
replace_abbreviations('я не оч, все раб. время в it отдыхаю, чил, зе бест')

'я не оч, все работа время в информационные технологии отдыхаю, чил, зе отлично'

In [586]:
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel

In [587]:
tokenizer = RegexTokenizer()
model = FastTextSocialNetworkModel(tokenizer=tokenizer)

In [588]:
def get_keys(list_of_dicts):
    keys = []
    for d in list_of_dicts:
        for key in d:
            keys.append(key)
    return keys

In [589]:
messages = df_all['answer'].values
list_of_sentiments = model.predict(messages, k=1)
df_all['sentiment'] = [sentiment if sentiment != 'speech' else 'positive' for sentiment in get_keys(list_of_sentiments)]

In [592]:
df_all['corrected'] = list(map(lambda answer: replace_abbreviations(answer), df_all['answer'].values))

In [594]:
messages = df_all['corrected'].values
list_of_sentiments = model.predict(messages, k=1)
df_all['sentiment_corr'] = [sentiment if sentiment != 'speech' else 'positive' for sentiment in get_keys(list_of_sentiments)]

In [606]:
model.predict(['подходы к лидерству'], k=1)

[{'neutral': 0.880807101726532}]

In [599]:
df_all[df_all['sentiment'] != df_all['sentiment_corr']][['question','answer','sentiment','corrected','sentiment_corr']]

,question,answer,sentiment,corrected,sentiment_corr
360,Ваша главная эмоции сейчас?,страшно чутка,skip,страшно немного,negative
520,Какие лучшие практики мотивации рабочих и ИТР ...,стабильная и достойная з/п,negative,стабильная и достойная зарплата,neutral
923,Ваши открытия за время обучения на модуле,подходы к лидерству,neutral,подходы к лидерство,negative
1458,Как ваше настроение?,#росатомсила,skip,крутая госкорпорация росатом сила,positive
1593,Как ваше настроение?,гуд,neutral,хорошо,positive
1622,Как ваше настроение?,бамбалейла,skip,отлично,positive
1945,Слово дня,росатом,neutral,крутая госкорпорация росатом,positive
1961,Основная мысль по результатам конференции,росатом прогрессирует,neutral,крутая госкорпорация росатом прогрессирует,positive
1973,Основная мысль по результатам конференции,росатом - люди,skip,крутая госкорпорация росатом - люди,neutral
2020,Основная мысль по результатам конференции,росатом - лидер,skip,крутая госкорпорация росатом - лидер,positive


In [596]:
from spellchecker import SpellChecker

spell = SpellChecker(language='ru')

In [597]:
print(spell.candidates('оформл'))

{'формы', 'форма', 'форму', 'форме'}
